In [10]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import *
from tkinter.ttk import *
import os
import time

#for finding data and necessary parameters

#set some variables needed in next dialog box
propDiameter = 10.5

selectAltitude = True         #global variable, False means use Density 

altitude = 0   #in feet
h = altitude/3.281 #convert to meters
Temperature = 15+273.15-6.5*h/1000 #temp in Kelvin
T = Temperature 
density = 1.225

#throttle information
Throttle = 100       #throttle value from 0 to 100

#esc information
eCurrMax = 12     #max current of esc
eRes = 1      #esc resistance in milliohms

#battery cell information --- specified from Li-Pos
cells = 3           #number of cells in Li-Po battery
battVolt = 3.7      #Li-Po per-cell Voltage
battRes = 0.022    #battery Resistance in milliohms

#motor information
kV = 1500            #RPM per volt for motor 
nLCurr = 0.3        #no load current in amps
nLVolt = 10          #no load voltage in Volts
mRes = 15         #motor resistance in milliohms
mCurr = 13           #max current of motor
    
#find file

#found online
def get_download_path():
    """Returns the default downloads path for linux or windows"""
    if os.name == 'nt':
        import winreg
        sub_key = r'SOFTWARE\Microsoft\Windows\CurrentVersion\Explorer\Shell Folders'
        downloads_guid = '{374DE290-123F-4565-9164-39C4925E467B}'
        with winreg.OpenKey(winreg.HKEY_CURRENT_USER, sub_key) as key:
            location = winreg.QueryValueEx(key, downloads_guid)[0]
        return location
    else:
        return os.path.join(os.path.expanduser('~'), 'downloads')

directory = get_download_path()
filename = ""


class Counter_program():
    def __init__(self):
        self.window = tk.Tk()
        self.window.title("Motor Calculator")
        
        style = ttk.Style()
        style.configure("BW.TLabel", foreground="black", background="white")
        
        #for changing style of elements in .ttk
        #s = ttk.Style()
        #s.configure('.', background='SteelBlue1')
        
        # Create some room around all the internal frames
        self.window['padx'] = 5
        self.window['pady'] = 5
        
        #for updating name of file under button
        def updateFileName(filename):
            global fileLabel
            fileLabel.configure(text = filename)
            
        #set filename to global
        global filename
        
        #for button to call file
        def callback():
            global filename
            filename = filedialog.askopenfilename(initialdir =  directory, title = "Select A File", filetype =((".dat files","*.dat"),("all files","*.*")) )
            fileLabel()
        
        # - - - - - - - - - - - - - - - - - - - - -
        # Menus
        menubar = tk.Menu(self.window)

        filemenu = tk.Menu(menubar, tearoff=0)
        filemenu.add_command(label="Open", command=callback)
        
        filemenu.add_separator()
        filemenu.add_command(label="Exit", command=self.window.destroy)
        menubar.add_cascade(label="File", menu=filemenu)

        self.window.config(menu=menubar)
        
        #default unit color
        unitColor = "slategrey"
        boxWidth = 5
        
        # - - - - - - - - - - - - - - - - - - - - -
        # Prop File Selector / Confirmer Window
        propeller_frame = ttk.LabelFrame(self.window, text="Propeller", relief=tk.RIDGE)
        propeller_frame.grid(row=1, column=1, sticky=tk.E + tk.W + tk.N + tk.S)
        

        #browse button for propeller data file
        button_label = ttk.Button(propeller_frame, text="Browse a File", command=callback)
        button_label.grid(row=1, column=1, sticky=tk.W, pady=3)
        
        print(filename)
        fileLabel = ttk.Label(propeller_frame, text = "No File Selected")
        fileLabel.grid(column = 1, row = 2,ipadx = 100)
        
        #for updating fileLabel below button
        def fileLabel():
            fileLabel = ttk.Label(propeller_frame, text = "")
            fileLabel.grid(column = 1, row = 2)
            fileLabel.configure(text = filename) 
        
        #propeller diameter
        propellerDiameter_label = ttk.Label(propeller_frame, text="Propeller Diameter")
        propellerDiameter_label.grid(row=3, column=1, sticky=tk.W + tk.N +tk.S)
        
        propellerDiameter_Units = ttk.Label(propeller_frame, text="inches",foreground=unitColor)
        propellerDiameter_Units.grid(row=3, column=3, sticky=tk.W)

        propellerDiameter_entry = ttk.Entry(propeller_frame, width=boxWidth)
        propellerDiameter_entry.grid(row=3, column=2, sticky=tk.W, pady=3)
        propellerDiameter_entry.insert(tk.END, "10")

        # - - - - - - - - - - - - - - - - - - - - -
        # Motor Parameters
        motor_frame = ttk.LabelFrame(self.window, text="Motor",relief=tk.RIDGE)
        motor_frame.grid(row=2, column=1, sticky=tk.E + tk.W + tk.N + tk.S)

        #kV code
        kV_label = ttk.Label(motor_frame, text="Motor kV")
        kV_label.grid(row=1, column=1, sticky=tk.W + tk.N +tk.S)
        
        kV_Units = ttk.Label(motor_frame, text="RPM/Volt",foreground=unitColor)
        kV_Units.grid(row=1, column=3, sticky=tk.W)
        
        kV_entry = ttk.Entry(motor_frame, width=boxWidth)
        kV_entry.grid(row=1, column=2, sticky=tk.W, pady=3)
        kV_entry.insert(tk.END, "1500")
        
        #Motor Resistance Code
        motorResistance_label = ttk.Label(motor_frame, text="Motor Resistance")
        motorResistance_label.grid(row=2, column=1, sticky=tk.W + tk.N +tk.S)
        
        motorResistance_Units = ttk.Label(motor_frame, text=u"m\u2126",foreground=unitColor)
        motorResistance_Units.grid(row=2, column=3, sticky=tk.W)

        motorResistance_entry = ttk.Entry(motor_frame, width=boxWidth)
        motorResistance_entry.grid(row=2, column=2, sticky=tk.W, pady=3)
        motorResistance_entry.insert(tk.END, "15")
        
        #Motor No Load Current
        motorNoLoadCurrent_label = ttk.Label(motor_frame, text="No Load Current")
        motorNoLoadCurrent_label.grid(row=3, column=1, sticky=tk.W + tk.N +tk.S)
        
        motorNoLoadCurrent_Units = ttk.Label(motor_frame, text="Amps",foreground=unitColor)
        motorNoLoadCurrent_Units.grid(row=3, column=3, sticky=tk.W)

        motorNoLoadCurrent_entry = ttk.Entry(motor_frame, width=boxWidth)
        motorNoLoadCurrent_entry.grid(row=3, column=2, sticky=tk.W, pady=3)
        motorNoLoadCurrent_entry.insert(tk.END, ".04")
        
        #Motor No Load Voltage (not used in calculations)
        motorNoLoadVoltage_label = ttk.Label(motor_frame, text="No Load Voltage")
        motorNoLoadVoltage_label.grid(row=4, column=1, sticky=tk.W + tk.N +tk.S)
        
        motorNoLoadVoltage_Units = ttk.Label(motor_frame, text="Volts",foreground=unitColor)
        motorNoLoadVoltage_Units.grid(row=4, column=3, sticky=tk.W)

        motorNoLoadVoltage_entry = ttk.Entry(motor_frame, width=boxWidth)
        motorNoLoadVoltage_entry.grid(row=4, column=2, sticky=tk.W, pady=3)
        motorNoLoadVoltage_entry.insert(tk.END, "5")
        
        #-------------------------------------------
        # ESC parameters
        esc_frame = ttk.LabelFrame(self.window, text="ESC",relief=tk.RIDGE)
        esc_frame.grid(row=3, column=1, sticky=tk.E + tk.W + tk.N + tk.S)

        #max current code
        ESCcurrent_label = ttk.Label(esc_frame, text="Max Current")
        ESCcurrent_label.grid(row=1, column=1, sticky=tk.W + tk.N +tk.S)
        
        ESCcurrent_Units = ttk.Label(esc_frame, text="Amps",foreground=unitColor)
        ESCcurrent_Units.grid(row=1, column=3, sticky=tk.W)

        ESCcurrent_entry = ttk.Entry(esc_frame, width=boxWidth)
        ESCcurrent_entry.grid(row=1, column=2, sticky=tk.W, pady=3)
        ESCcurrent_entry.insert(tk.END, "160")
        
        #esc resistance code
        ESCresistance_label = ttk.Label(esc_frame, text="ESC Resistance")
        ESCresistance_label.grid(row=2, column=1, sticky=tk.W + tk.N +tk.S)
        
        ESCresistance_Units = ttk.Label(esc_frame, text=u"m\u2126",foreground=unitColor)
        ESCresistance_Units.grid(row=2, column=3, sticky=tk.W)

        ESCresistance_entry = ttk.Entry(esc_frame, width=boxWidth)
        ESCresistance_entry.grid(row=2, column=2, sticky=tk.W, pady=3)
        ESCresistance_entry.insert(tk.END, "1")
    
        #-------------------------------------------
        # Battery
        battery_frame = ttk.LabelFrame(self.window, text="Battery",relief=tk.RIDGE)
        battery_frame.grid(row=2, column=2, sticky=tk.E + tk.W + tk.N, padx=6)

        #Cell Count
        cellCount_label = ttk.Label(battery_frame, text="Cell Count")
        cellCount_label.grid(row=1, column=1, sticky=tk.W + tk.N +tk.S)
        
        cellCount_Units = ttk.Label(battery_frame, text="cells",foreground=unitColor)
        cellCount_Units.grid(row=1, column=3, sticky=tk.W)

        cellCount_entry = ttk.Entry(battery_frame, width=boxWidth)
        cellCount_entry.grid(row=1, column=2, sticky=tk.W, pady=3)
        cellCount_entry.insert(tk.END, "4")
        
        #Voltage per Cell
        cellVoltage_label = ttk.Label(battery_frame, text="Voltage Per Cell")
        cellVoltage_label.grid(row=2, column=1, sticky=tk.W + tk.N +tk.S)
        
        cellVoltage_Units = ttk.Label(battery_frame, text="Volts",foreground=unitColor)
        cellVoltage_Units.grid(row=2, column=3, sticky=tk.W)

        cellVoltage_entry = ttk.Entry(battery_frame, width=boxWidth)
        cellVoltage_entry.grid(row=2, column=2, sticky=tk.W, pady=3)
        cellVoltage_entry.insert(tk.END, "3.7")
        
        ''' can also use C rating but wanted to keep it simple for now
        #mAh Rating
        mAh_label = ttk.Label(battery_frame, text="mAh Rating")
        mAh_label.grid(row=3, column=1, sticky=tk.W + tk.N +tk.S)
        
        mAh_Units = ttk.Label(battery_frame, text= "mAh",foreground=unitColor)
        mAh_Units.grid(row=3, column=3, sticky=tk.W)

        mAh_entry = ttk.Entry(battery_frame, width=boxWidth)
        mAh_entry.grid(row=3, column=2, sticky=tk.W, pady=3)
        mAh_entry.insert(tk.END, "4500")
        
        #C-rating
        cRating_label = ttk.Label(battery_frame, text="C Rating")
        cRating_label.grid(row=4, column=1, sticky=tk.W + tk.N +tk.S)
        
        cRating_Units = ttk.Label(battery_frame, text="C",foreground=unitColor)
        cRating_Units.grid(row=4, column=3, sticky=tk.W)

        cRating_entry = ttk.Entry(battery_frame, width=boxWidth)
        cRating_entry.grid(row=4, column=2, sticky=tk.W, pady=3)
        cRating_entry.insert(tk.END, "20")
        '''
        
        #iR
        batteryResistance_label = ttk.Label(battery_frame, text="Battery IR")
        batteryResistance_label.grid(row=5, column=1, sticky=tk.W + tk.N +tk.S)
        
        batteryResistance_Units = ttk.Label(battery_frame, text=u"m\u2126",foreground=unitColor)
        batteryResistance_Units.grid(row=5, column=3, sticky=tk.W)

        batteryResistance_entry = ttk.Entry(battery_frame, width=boxWidth)
        batteryResistance_entry.grid(row=5, column=2, sticky=tk.W, pady=3)
        batteryResistance_entry.insert(tk.END, "15")
        
        # - - - - - - - - - - - - - - - - - - - - -
        # The Throttle frame
        throttle_frame = ttk.LabelFrame(self.window, text="Throttle", relief=tk.RIDGE)
        throttle_frame.grid(row=1, column=2, sticky=tk.E + tk.W + tk.N + tk.S, padx=6)
        
        throttle_scale = tk.Scale(throttle_frame, from_=0, to=100, digits = 100, orient=tk.HORIZONTAL,width=8, length=200)
        throttle_scale.grid(row=1, column=1, sticky=tk.W + tk.N)
        throttle_scale.set(100)
        
        # - - - - - - - - - - - - - - - - - - - - -
        # The Environment Frame
        environment_frame = ttk.LabelFrame(self.window, text="Environment", relief=tk.RIDGE)
        environment_frame.grid(row=3, column=2, sticky=tk.E + tk.W + tk.N + tk.S, padx=6)
        
        entry_label = ttk.Label(environment_frame, text="Select Altitude or Density")
        entry_label.grid(row=1, rowspan=3, column=1, sticky=tk.W + tk.N)
        
        altitudeDensity_entry = ttk.Entry(environment_frame, width=boxWidth)
        altitudeDensity_entry.grid(row=4, column=2, sticky=tk.W, pady=3)
        altitudeDensity_entry.insert(tk.END, "15")
        
        def sel():
            #Density
            global selectAltitude
            selectAltitude = False
            density_label = ttk.Label(environment_frame, text="Density")
            density_label.grid(row=4, column=1, sticky=tk.W + tk.N + tk.S)
        
            density_Units = ttk.Label(environment_frame, text="kg/m\N{SUPERSCRIPT THREE}",foreground=unitColor)
            density_Units.grid(row=4, column=3, sticky=tk.W + tk.E)
            
                    
        def sel2():
        #Altitude
            global selectAltitude 
            selectAltitude = True
            altitude_label = ttk.Label(environment_frame, text="Altitude")
            altitude_label.grid(row=4, column=1, sticky=tk.W + tk.N + tk.S)
        
            altitude_Units = ttk.Label(environment_frame, text="feet    ",foreground=unitColor)
            altitude_Units.grid(row=4, column=3, sticky=tk.W + tk.E)
            
        #select altitude to make sure something is selected
        sel2()
        self.radio_variable = tk.StringVar()
        self.radio_variable.set("0")

        Altitude_selected = ttk.Radiobutton(environment_frame, text="Altitude",variable=self.radio_variable, value="0",command=sel2)
        Density_selected = ttk.Radiobutton(environment_frame, text="Density",variable=self.radio_variable, value="1",command=sel)
        
        Altitude_selected.grid(row=2, column=2, sticky=tk.W)
        Density_selected.grid(row=3, column=2, sticky=tk.W)
        
        #now set all global variables from entries - update function
        def update():
            global filename
            global selectAltitude 
            global propDiameter
            global altitude
            global density
            global eCurrMax
            global eRes
            global cells 
            global battVolt 
            global battRes
            global kV 
            global nLCurr
            global nLVolt
            global mRes
            global mCurr 
            global Throttle
            propDiameter = propellerDiameter_entry.get()
            if(selectAltitude):
                altitude = altitudeDensity_entry.get()
            elif(selectAltitude == False):
                density = altitudeDensity_entry.get()
            eCurrMax = ESCcurrent_entry.get()
            eRes = ESCresistance_entry.get()
            cells = cellCount_entry.get()
            battVolt = cellVoltage_entry.get()
            battRes = batteryResistance_entry.get()
            kV = kV_entry.get()
            nLCurr = motorNoLoadVoltage_entry.get()
            nLVolt
            mRes = motorResistance_entry.get()
            mCurr = motorNoLoadCurrent_entry.get()
            Throttle = throttle_scale.get()
            
            #for creating an excel sheet with all properties to verify 
            propertiesTable = ['File Name','Propeller Diameter', 'Altitude', 'Density', 'ESC Max Current', 
                               'ESC Resistance', 'Cell Count', 'Battery Voltage', 'Battery Resistance', 'Motor kV', 
                               'Motor No Load Current', 'Motor No Load Voltage', 'Motor Resistance', 'Throttle']
            valuesTable = [filename,propDiameter,altitude,density,eCurrMax,eRes,cells, battVolt, battRes,
                           kV,nLCurr,nLVolt,mRes,Throttle]
            
            #remove either density or altitude to avoid confusion
            if(selectAltitude):
                propertiesTable.remove('Density')
                del valuesTable[3]
            else:
                propertiesTable.remove('Altitude')
                del valuesTable[2]
                
            #create dataframe, export to excel
            dfProperties = pd.DataFrame({'Properties': propertiesTable, 'Values': valuesTable})
            #print(dfProperties)
            dfProperties.to_excel('propellerdata' + str(propDiameter) + '.xlsx', sheet_name='sheet1', index=False)
            
        # Finish button in the lower right corner
        finish_button = ttk.Button(self.window, text = "Submit Input", command=lambda:[update(),self.window.destroy()])
        finish_button.grid(row=4, column=2)
        
    def exit(self):
        self.frame.destroy()
        
        # - - - - - - - - - - - - - - - - - - - - -

# Create the entire GUI program
program = Counter_program()

# Start the GUI event loop
program.window.mainloop()



#print(selectAltitude)
#print(propDiameter)
#print(altitude)
#print(density)
#print(eCurrMax)
#print(eRes)      
#print(cells)           
#print(battVolt)
#print(battRes)
#print(kV)
#print(nLCurr)
#print(nLVolt)
#print(mRes)
#print(mCurr)
#print(Throttle)


In [11]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import *
from tkinter.ttk import *
import os

#for accessing data sorting data


#set some variables needed in next dialog box
filename = 'C:/Users/suzuk/Downloads/PERFILES_WEB-202006/PERFILES2/PER3_16x10E.dat'

propDiameter = 20

selectAltitude = True         #global variable, False means use Density 

altitude = 0   #in feet
h = altitude/3.281 #convert to meters
Temperature = 15+273.15-6.5*h/1000 #temp in Kelvin
T = Temperature 
density = 1.225

#throttle information
Throttle = 100       #throttle value from 0 to 100

#esc information
eCurrMax = 120     #max current of esc
eRes = 0.004      #esc resistance in ohms

#battery cell information --- specified from Li-Pos
cells = 6           #number of cells in Li-Po battery
battVolt = 3.7      #Li-Po per-cell Voltage
battRes = 0.0028    #battery Resistance in ohms

#motor information
kV = 295            #RPM per volt for motor 
nLCurr = 1.7        #no load current in amps
nLVolt = 8.4        #no load voltage in Volts
mRes = 0.015         #motor resistance in ohms
mWatt = 2000          #max watts of motor
    
file = 'PER3_20x10E'
             #force file name
#filename = 'C:\\Users\\suzuk\\Downloads\\' + file + '.dat.txt'
filename = 'C:\\Users\\marym\\Desktop\\PropellerCalc\\propellers\\' + file + '.dat'

#print(filename)   #print filename if having problems

#set column index to help import data
header_list =  [str(x) for x in range(12 + 1)]

#read in file
propdata = pd.read_csv(filename, sep='\s+', names=header_list)
propdata = propdata.to_numpy()
df = pd.DataFrame(propdata)
df.to_csv(index=False)

#find all rows with rpm data
searchString = "RPM"
mask = np.column_stack([df[col].str.contains(r''+searchString, na=False) for col in df])
RPMdata = pd.DataFrame.dropna(df.loc[mask.any(axis=1)],axis=1)

#create new table called RPM values to be able to bin RPM values into groups
#remove characters from PROP RPM and change to numeric values to make analysis easier
#first find column with numbers by converting all columns to numbers
RPMs = RPMdata.apply(pd.to_numeric,errors = 'ignore')

#then remove all non-int64 columns
RPMs = RPMs.select_dtypes(include=['int64'])

#get first RPM value index
a = RPMs.columns.values           #weird name, maybe switch later, a[] is for locating which column RPM values at
indexminRPM = RPMs[a[0]].index[0]

#delete unnecessary top rows of RPM data using first RPM value
RPMdf = df.iloc[indexminRPM:]

#get max RPM and index
maxRPM = RPMs.max()
indexmaxRPM = RPMs[a[0]].idxmax()

#get index of each RPM value to bin data
RPMindex = RPMs.index

#clear out columns with mostly all NA
RPMdf = pd.DataFrame.dropna(RPMdf,axis='columns',thresh=10)

#add index of last row of database to RPM dataframe to bin correctly
lastIndex = RPMdf.iloc[[-1]].index.values[0]

#get index of RPMs to group them by their RPM
RPMindex = np.append(RPMindex,lastIndex)
RPMlabel = RPMs[a[0]].to_list()

#label columns as what they're portraying
#assumes all bins of RPM have same values at same columns
#assumes V is in first column, find it
Units = RPMdf[RPMdf[0].str.match('V')]

#find first row V is in
unitsRow = Units[0].index[0]

#take first row and set this as column headers
RPMdf.columns = RPMdf.loc[unitsRow]
    
#remove all non-numeric rows
RPMdf = RPMdf.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

#finally group each RPM
RPMdf = RPMdf.groupby(pd.cut(RPMdf.index, RPMindex, right=False, labels = RPMlabel))

#for printing out grouped dataframe
#for name,group in RPMdf:
#   print(name)
#   print(group)
       

SyntaxError: EOL while scanning string literal (Temp/ipykernel_8740/2507062655.py, line 14)

In [12]:
import numpy as np
import pandas as pd
import tkinter as tk
from tkinter import ttk
from tkinter import filedialog
from tkinter import *
from tkinter.ttk import *
import os

#for accessing data sorting data


#set some variables needed in next dialog box
filename = 'C:/Users/suzuk/Downloads/PERFILES_WEB-202006/PERFILES2/PER3_20x10E.dat'

propDiameter = 20

selectAltitude = True         #global variable, False means use Density 

altitude = 0   #in feet
h = altitude/3.281 #convert to meters
Temperature = 15+273.15-6.5*h/1000 #temp in Kelvin
T = Temperature 
density = 1.225

#throttle information
Throttle = 100       #throttle value from 0 to 100

#esc information
eCurrMax = 120     #max current of esc
eRes = 0.004      #esc resistance in ohms

#battery cell information --- specified from Li-Pos
cells = 6           #number of cells in Li-Po battery
battVolt = 3.7      #Li-Po per-cell Voltage
battRes = 0.0028    #battery Resistance in ohms

#motor information
kV = 295            #RPM per volt for motor 
nLCurr = 1.7        #no load current in amps
nLVolt = 8.4        #no load voltage in Volts
mRes = 0.015         #motor resistance in ohms
mWatt = 2000          #max watts of motor
    
file = 'PER3_20x10E'
             #force file name
#filename = 'C:\\Users\\suzuk\\Downloads\\' + file + '.dat.txt'
filename = 'C:\\Users\\suzuk\\Desktop\\PropellerCalc\\propellers\\' + file + '.dat'

#print(filename)   #print filename if having problems

#set column index to help import data
header_list =  [str(x) for x in range(12 + 1)]

#read in file
propdata = pd.read_csv(filename, sep='\s+', names=header_list)
propdata = propdata.to_numpy()
df = pd.DataFrame(propdata)
df.to_csv(index=False)

#find all rows with rpm data
searchString = "RPM"
mask = np.column_stack([df[col].str.contains(r''+searchString, na=False) for col in df])
RPMdata = pd.DataFrame.dropna(df.loc[mask.any(axis=1)],axis=1)

#create new table called RPM values to be able to bin RPM values into groups
#remove characters from PROP RPM and change to numeric values to make analysis easier
#first find column with numbers by converting all columns to numbers
RPMs = RPMdata.apply(pd.to_numeric,errors = 'ignore')

#then remove all non-int64 columns
RPMs = RPMs.select_dtypes(include=['int64'])

#get first RPM value index
a = RPMs.columns.values           #weird name, maybe switch later, a[] is for locating which column RPM values at
indexminRPM = RPMs[a[0]].index[0]

#delete unnecessary top rows of RPM data using first RPM value
RPMdf = df.iloc[indexminRPM:]

#get max RPM and index
maxRPM = RPMs.max()
indexmaxRPM = RPMs[a[0]].idxmax()

#get index of each RPM value to bin data
RPMindex = RPMs.index

#clear out columns with mostly all NA
RPMdf = pd.DataFrame.dropna(RPMdf,axis='columns',thresh=10)

#add index of last row of database to RPM dataframe to bin correctly
lastIndex = RPMdf.iloc[[-1]].index.values[0]

#get index of RPMs to group them by their RPM
RPMindex = np.append(RPMindex,lastIndex)
RPMlabel = RPMs[a[0]].to_list()

#label columns as what they're portraying
#assumes all bins of RPM have same values at same columns
#assumes V is in first column, find it
Units = RPMdf[RPMdf[0].str.match('V')]

#find first row V is in
unitsRow = Units[0].index[0]

#take first row and set this as column headers
RPMdf.columns = RPMdf.loc[unitsRow]
    
#remove all non-numeric rows
RPMdf = RPMdf.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()

#finally group each RPM
RPMdf = RPMdf.groupby(pd.cut(RPMdf.index, RPMindex, right=False, labels = RPMlabel))

#for printing out grouped dataframe
#for name,group in RPMdf:
#   print(name)
#   print(group)
       

#NOTES#
#J = Velocity / (RPs*Diameter (m))
"""
Some Shortened terms

Curr = Current
Res = Resistance
Batt = Battery
Volt = Voltage
v = Voltage (when put before a term)
p = Power (when put before a term)
Amps = Amperage
"""
propDiameter = float(propDiameter)    #in inches
altitude = float(altitude)            #in feet
density = float(density)              #in kg / m^3
eCurrMax = float(eCurrMax)            #in amps
eRes = float(eRes)                    #in ohms
cells = float(cells)                  #integer
battVolt = float(battVolt)            #in volts
battRes = float(battRes)              #in ohms
kV = float(kV)                        #in RPM/volt
nLCurr = float(nLCurr)                #in amps
nLVolt = float(nLVolt)                #in volts
mRes = float(mRes)                    #in ohms
mWatt = float(mWatt)                  #in watts
Throttle = float(Throttle)            #scale from 0 to 100

#""" for debugging
print(selectAltitude)
print('propDiameter: '+str(propDiameter))
print('altitude: '+str(altitude))
print('density: '+str(density))
print('eCurrMax: '+str(eCurrMax))
print('eRes: '+str(eRes))      
print('battRes: '+str(battRes))           
print('battVolt: '+str(battVolt))
print('cells: '+str(cells))
print('kV: '+str(kV))
print('nLCurr: '+str(nLCurr))
#print('nLVolt: '+str(nLVolt))
print('mRes: '+str(mRes))
print('mWatt: '+str(mWatt))
print('Throttle: '+str(Throttle))
#"""

propDiameter = propDiameter*0.0254     #convert prop diameter to meters

#WARNING --- pressure and altitude calcaulator only valid to approximately 12,000 m
if(selectAltitude):
    h = altitude/3.281 #convert to meters
    Temperature = 15+273.15-6.5*h/1000 #temp in Kelvin
    T = Temperature 
    R = 8.3143       #constant
    M = 0.02896      #molar mass of Earth's air
    g = 9.806        #gravity
    Po = 101325      #sea level pressure in kPa
    Pressure = Po*np.exp(-M*g*h/(R*T))    #obtained Pressure in kPa
    Density = (Pressure*M)/(R*T)          #unit is in kg/m^3
else:
    Density = density

#------------------all these can be forced but are selected by user in above code--------------
#throttle information force 

#propDiameter = 0.508  #prop Diameter in meters
#Throttle = 100       #throttle value from 0 to 100

#esc information
#eCurrMax = 160     #max current of esc
#eRes = 0.001      #esc resistance in ohms

#battery cell information --- specified from Li-Pos
#cells = 6           #number of cells in Li-Po battery
#battVolt = 3.7      #Li-Po per-cell Voltage
#battRes = 0.012    #battery Resistance in ohms

#motor information
#kV = 295            #RPM per volt for motor 
#nLCurr = 1.7        #no load current in amps
#nLVolt = 10          #no load voltage in Volts
#mRes = 0.015         #motor resistance in ohms
#mCurr = 200         #max current of motor

#-----------------------------------------------------------------------------------------------

#Note ---- Prof. Merret at UIUC helped provide these formulas, comments added by me for the most part to try to explain each term

RPMgroups = np.array(list(RPMdf.groups))
allAmps = []
for groups in RPMgroups:
    RPMgroup = RPMdf.get_group(groups)
    PWR = RPMgroup['PWR'].iloc[0]*745.7
    allAmps.append(PWR/(cells*battVolt))
#allAmps is an array with expected amp draw at max static thrust at each velocity 
#print(allAmps)
velocityArray = [0]
velocityStep = 1.524

def RPMfinder(motorAmpGuess,battAmpGuess,Output,velocity):
    
    #main calulation for iteration of pMotorOut
    def pMotorOutCalc(motorAmp,battAmp):
        
        vBatt = cells*battVolt-cells*battRes*battAmp   #get total battery volts with voltage loss
        vTerm = (vBatt-eRes*battAmp)*Throttle/100      #get voltage drop from esc and throttle value
        vDropMotor = motorAmp*mRes                     #get voltage drop from motor
        pMotorIn = vTerm*battAmp                       #get power motor uses in operation
        vEMF = vTerm-vDropMotor                        #get voltage motor actually has that after esc,battery,throttle losses
        pMotorOut = vEMF*(motorAmp-nLCurr)             #get power motor generates (voltage of motor*(Amps of motor-no load motor Amps))
        pMotorIn = vTerm*(motorAmp-nLCurr)             #get power motor receives
        
        return pMotorIn,pMotorOut,vTerm,motorAmp,vBatt,battAmp,vEMF
    
    pMotorIn,pMotorOut,vTerm,motorAmp,vBatt,battAmp,vEMF = pMotorOutCalc(motorAmpGuess,battAmpGuess)
        
    RPM = vEMF*kV                                  #get revolutions per minute of motor
    RPS = RPM/60                                   #get revolutions per second of motor
    #print('RPS: '+str(RPS))
    
    '''
    #checks if battery can supply current
    if(battAmp > battVolt/battRes):
        raise Exception("Batteries can't supply this much current: " + str(battAmp))
        
    #checks if ESC can handle Current
    if(battAmpGuess > eCurrMax):
        raise Exception("ESC will be overloaded, choose larger ESC: " + str(Amps))
    '''   
        
    #checks if RPM is positive    
    if(RPM < 0):                  
        raise Exception("RPM is negative, something is wrong: " + str(RPM))
        
    #find closest data to RPM values, both above and below it
    RPMgroups = np.array(list(RPMdf.groups))
    if(RPM > RPMgroups.max()):
        raise Exception('RPM maximum limit reached, no data for higher RPM')
    if(RPM < RPMgroups.min()):
        raise Exception('RPM minimum limit reached, no data for lower RPM')
    bottomRPM = RPMgroups[RPMgroups < RPM].max()
    topRPM = RPMgroups[RPMgroups > RPM].min()

    #then get bottom and top RPM groups
    bottomRPMdata = RPMdf.get_group(bottomRPM).reset_index()
    topRPMdata = RPMdf.get_group(topRPM).reset_index()
    
    #then get cp and ct of both bottom and top RPM groups for desired velocity
    velocityMPH = velocity*2.23694
    
    #print('Velocity: ' + str(velocityMPH))
    #print('Index Velocity: ' + str(abs(bottomRPMdata['V']-velocityMPH).idxmin()))
    #print(bottomRPMdata)
    #find relevant indexes
    bottomRPMClosestVelocity = abs(bottomRPMdata['V']-velocityMPH).idxmin()
    topRPMClosestVelocity = abs(topRPMdata['V']-velocityMPH).idxmin()
    
    #find relevant ct
    bottomct = bottomRPMdata['Ct'].iloc[bottomRPMClosestVelocity]
    topct = topRPMdata['Ct'].iloc[topRPMClosestVelocity]
    
    #find relevant cp
    bottomcp = bottomRPMdata['Cp'].iloc[bottomRPMClosestVelocity]
    topcp = topRPMdata['Cp'].iloc[topRPMClosestVelocity]
    
    #find relevant torques
    bottomTorque = bottomRPMdata['Torque'].iloc[bottomRPMClosestVelocity]
    topTorque = topRPMdata['Torque'].iloc[topRPMClosestVelocity]
    
    '''
    %works for 0 m's
    bottomct = bottomRPMdata['Ct'].iloc[0]
    topct = topRPMdata['Ct'].iloc[0]
    bottomcp = bottomRPMdata['Cp'].iloc[0]
    topcp = topRPMdata['Cp'].iloc[0]
    '''
    '''
    print('bottomct: ' +str(bottomct))
    print('topct: ' +str(topct))
    print('bottomcp: ' +str(bottomcp))
    print('topcp: ' +str(topcp))
    '''
    #interpolate to get cp and ct for motor RPM
    cp = bottomcp+(RPM-bottomRPM)*(topcp-bottomcp)/(topRPM-bottomRPM)
    ct = bottomct+(RPM-bottomRPM)*(topct-bottomct)/(topRPM-bottomRPM)
    torque = bottomTorque+(RPM-bottomRPM)*(topTorque-bottomTorque)/(topRPM-bottomRPM)
    
    
    #prop power and thrust calculation
    pProp = Density*RPS**3*propDiameter**5*cp   #get power prop requires in kg*m^2/s^3  (W)
    ThrustProp = Density*RPS**2*propDiameter**4*ct   #get thrust prop generates in kg*m/s^2  (N)

    '''
    print('Density: '+str(Density))
    print('Prop D: ' + str(propDiameter))
    print('Cp: '+str(cp))
    print('\n')
    print('pProp: '+str(pProp))
    print('Thrust Prop: ' + str(ThrustProp))
    print('\n')
    '''
    #in hP and english units
    #pPropEng = pProp*0.74                  #get power prop requires in ft-lb/s
    #pProphp = pProp*0.00134102             #get power prop requires in hp
    #tPropEng = ThrustProp*0.2248090795     #get thrust prop generates in lb
    
    if Output == 1:
        print('Export final parameters here')
    
    return pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,torque

#first Amp guess
ImotorGuess = nLCurr+0.01 #make first motor amp draw guess as no load amp draw for motor
IbattGuess = nLCurr+0.01


'''
pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF = RPMfinder(ImotorGuess,IbattGuess,0,0)  #run first iteration of RPM finder + pMotorOut

pESCout = pMotorIn
vESCin = cells*battVolt-cells*battRes*battAmp
vESCout = cells*battVolt-(cells*battRes+eRes)*battAmp
ImotorGuess = pProp/vEMF+nLCurr
IbattGuess = pESCout/vESCout
'''
'''
print('vESCin: '+str(vESCin))
print('vESCout: '+str(vESCout))
print('ImotorGuess: '+str(ImotorGuess))
print('IbattGuess: '+str(IbattGuess))
print('Motor Power: ' + str(pMotorOut) + ' W')
print('Thrust Prop: ' + str(ThrustProp) + ' N')
print('Prop Power: ' + str(pProp) + ' W')
print('pMotorIn: ' + str(pMotorIn) + ' W')
print('break\n')
''' 
#dataframe for storing final results
resultProp = pd.DataFrame(columns=[ "Battery Current (A)","RPM (rev/min)", "Thrust (N)","Power (W)","Efficiency", "Torque (in-lbf)"])
resultProp.columns.name = 'Velocity (m/s)'


powerTolerance = 0.01 #how close the power out of the motor should be to the power out of the propeller

velocityPosition = 0 #velocityArray position
velocity = velocityArray[velocityPosition] #current velocity in m/s

#have to run here for the first time to make while loop work
pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(ImotorGuess,IbattGuess,0,velocity)

while(ThrustProp>0):

    if(velocity != 0):
        velocityArray.append(velocity)
    
    timeRun = 0 #variable to determine how often loop has run
    
    #run first time here to verify while loop
    #print('Velocity: ' + str(velocity))
    pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(ImotorGuess,IbattGuess,0,velocity)
    
    '''
    print(pProp)
    print(pMotorOut)
    '''
    
    while(abs(pProp-pMotorOut)>powerTolerance):
        pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(ImotorGuess,IbattGuess,0,velocity)
        '''
        print('Motor Power: ' + str(pMotorOut) + ' W')
        print('Thrust Prop: ' + str(ThrustProp) + ' N')
        print('Prop Power: ' + str(pProp) + ' W')
        print('pMotorIn: ' + str(pMotorIn) + ' W')
        print('prop calc done\n')
        '''
        #checks if motor power out is positive
        if(pMotorOut < 0):
            raise Exception("Motor Power Out is negative, something is wrong")
        pESCOut = pMotorIn
        vESCin = cells*battVolt-cells*battRes*battAmp
        vESCout = cells*battVolt-(cells*battRes+mRes)*battAmp
        ImotorGuess = pProp/vEMF+nLCurr
        IbattGuess = pESCOut/vESCout
        """print('pESC: '+str(pESCOut))
        print('vESCin: '+str(vESCin))
        print('vESCout: '+str(vESCout))
        print('ImotorGuess: '+str(ImotorGuess))
        print('IbattGuess: '+str(IbattGuess))
        """
        timeRun = timeRun+1
        #print('Iteration: ' + str(timeRun) + '     Velocity: '+ str(velocityArray[velocityPosition])+' m\s')
    
    #run one more time and store results in dataframe
    pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBatt,battAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(ImotorGuess,IbattGuess,0,velocity)
    #can keep previous motorAmp and battAmp values as they will be close to new ones most likely
    
    #print('motorAmp: '+str(motorAmp))
    #print('battAmp: '+str(battAmp))
    
    efficiency = pMotorOut/pMotorIn
    
    resultProp.loc[velocityArray[velocityPosition]] = [motorAmp,RPM,ThrustProp,pProp,efficiency,Torque]
    
    velocityPosition = velocityPosition+1
    velocity = velocity + velocityStep
    
    #print('New Velocity')

thrustLB = resultProp.loc[:,'Thrust (N)']*0.224809
resultProp['Thrust (lbf)'] = thrustLB
#drop last row because FN is 0 so results may be invalid
resultProp.drop(resultProp.tail(1).index,inplace=True)
resultProp
resultProp.to_csv('twinMotor.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\suzuk\\Desktop\\PropellerCalc\\propellers\\PER3_20x10E.dat'